In [5]:
import requests
import json

#### Data import

In [ ]:
api_link = "https://api.elhub.no/energy-data/v0/consumption-groups"

response = requests.get(api_link)

if response.status_code == 200:
    if response.text:
        data = response.json()
        consumption_groups = [i["id"] for i in data["data"]]

household
cabin
primary
secondary
tertiary
industry
private
business
*


#### 1. Utvikling i strømforbruk
Hvordan har strømforbruket i **NO5** utviklet seg de siste årene blant de forskjellige forbrukergruppene?  
Har forbruksmønsteret endret seg siden 2021?